In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import json
import numpy as np

from collections import Counter

In [2]:
from folium.plugins import MarkerCluster

In [11]:
from sklearn.cluster import DBSCAN

In [3]:
MOW = [55.75222, 37.61556]

In [4]:
train_df = pd.read_csv('./data/train_set.csv')
mcc_codes = pd.read_csv('./data/mcc_codes.csv', index_col='mcc')
train_df.head()

C:\Anaconda3\work\raiffazen\env\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_adress_lat,pos_adress_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
0,2.884034,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177
1,2.775633,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177
2,3.708368,NaN,NaN,NaN,St Petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858198,30.229024,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177
3,2.787498,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177
4,2.892510,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177


In [5]:
customer_gr = train_df.groupby('customer_id')

In [6]:
test_cust = '0007297d86e14bd68bd87b1dbdefe302'
test_set = customer_gr.get_group(test_cust)

In [7]:
mcc_code = 5411
position = test_set.query('mcc == @mcc_code').loc[:,['pos_adress_lat', 'pos_adress_lon']].dropna().as_matrix()

home_pos = test_set.loc[:,['home_add_lat','home_add_lon']].dropna().as_matrix()
home_pos = np.unique(home_pos, axis=0)[0]

#fig = folium.element.Figure()
map_ = folium.Map(location=MOW, zoom_start=11)
сluster = MarkerCluster(position).add_to(map_)
сluster2 = MarkerCluster([home_pos], popups= ['Home']).add_to(map_)

map_

In [100]:
# test DBSCAN
        


core_samples_mask = np.zeros_like(dbs.labels_, dtype=bool)
core_samples_mask[dbs.core_sample_indices_] = True


unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = position[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = position[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

#plt.title('Estimated number of clusters: %d' % n_clusters_)
#plt.show()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 27 but corresponding boolean dimension is 65

In [195]:
def haversine(pos1, pos2):
    """
    Calculate the great circle distance between two points 
    """
    return (pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2

In [87]:
clust_size = 0
for label in set(labels):
    if label<0:
        continue
    clust = position[labels == label]
    
    if clust_size < clust.shape[0]:
        best_clust = np.mean(clust, axis = 0)
        clust_size = clust.shape[0]

print(best_clust, ' ', clust_size, ' ', haversine(best_clust, home_pos))

[55.80063222 37.40069227]   40   0.012796791318208038


In [175]:
def dbscan_clust(position, home_pos = np.nan, dbscan_params = {'leaf_size':5, 'eps': 0.02, 'n_jobs': 5}):
    dbs = DBSCAN(**dbscan_params)
    dbs.fit(position)
    labels = dbs.labels_

    clust_size = 0
    best_clust = [0, 0]
    clust_size = np.NAN
    d = np.Inf
    
    for label in set(labels):
        if label<0:
            continue
        clust = position[labels == label]

        if clust_size < clust.shape[0]:
            best_clust = np.mean(clust, axis = 0)
            clust_size = clust.shape[0]
            print(best_clust)
            d = haversine(best_clust, home_pos)
            
    return best_clust, clust_size, d

In [ ]:
customer_mcc_gr = train_df.groupby(['customer_id', 'mcc'])

In [240]:
dbscan_params = {'leaf_size':5, 'eps': 0.02, 'n_jobs': 5, 'min_samples': 2}
r = 0.02
r2 = r**2

def get_dbscan_score(customer_gr, dbscan_params, mcc_code = 5411, calc_dist = True):
    score = 0
    res = {}
    
    for test_cust, test_set in customer_gr:
        test_set = test_set.query('mcc == @mcc_code')
        position = test_set.loc[:,['pos_adress_lat', 'pos_adress_lon']].dropna().as_matrix()

        if calc_dist:
            home_pos = test_set.loc[:,['home_add_lat','home_add_lon']].dropna().as_matrix()
            if (len(home_pos) ==0):
                continue
            home_pos = np.unique(home_pos, axis=0)[0]
            if len(np.unique(home_pos, axis=0)) >2:
                raise ValueError('Wrong home position. Check it!!!')
        else:
            hom_pos = np.array([90, 90])

        dbs = DBSCAN(**dbscan_params)
        dbs.fit(position)
        labels = dbs.labels_

        clust_size = -1
        d = np.Inf
        best_clust = np.array([90, 90])
        
        for label in set(labels):
            if label<0:
                continue
            clust = position[labels == label]
            if clust_size < clust.shape[0]:
                best_clust = np.mean(clust, axis = 0)
                clust_size = clust.shape[0]
                if calc_dist:
                    d = haversine(best_clust, home_pos)
                else:
                    d = np.inf
                    
        res.update({test_cust: [best_clust[0], best_clust[1], home_pos[0], home_pos[1], d, clust_size]})
        
    res_df = pd.DataFrame.from_dict(res,  orient='index')
    res_df.columns = ['best_post_lon', 'best_post_lat',  'home_post_lon', 'home_post_lat', 'dist', 'clust_size']
        
    return res_df

In [238]:
eps_list = np.logspace(-2, -1, 5)
min_samples_list = [5, 10, 15, 20, 25, 30]

res = {}

for min_samples in min_samples_list:
    for eps in eps_list: 
        res_df = get_dbscan_score(customer_gr, {'leaf_size':5, 'eps': eps, 'min_samples': min_samples})
        score = res_df.query('dist < @r2').shape[0]
        print(f'{eps}, {min_samples}, {score}')
        res.update({(eps, min_samples,):score})

0.01, 5, 2444
0.01778279410038923, 5, 2554
0.03162277660168379, 5, 2471
0.05623413251903491, 5, 2217
0.1, 5, 1850
0.01, 10, 1950
0.01778279410038923, 10, 2070
0.03162277660168379, 10, 2044
0.05623413251903491, 10, 1855
0.1, 10, 1564
0.01, 15, 1570
0.01778279410038923, 15, 1671
0.03162277660168379, 15, 1647
0.05623413251903491, 15, 1522
0.1, 15, 1287
0.01, 20, 1276
0.01778279410038923, 20, 1394
0.03162277660168379, 20, 1391
0.05623413251903491, 20, 1280
0.1, 20, 1100
0.01, 25, 1051
0.01778279410038923, 25, 1159
0.03162277660168379, 25, 1175
0.05623413251903491, 25, 1095
0.1, 25, 932


KeyboardInterrupt: 

In [242]:
eps_list = np.logspace(-3, -1, 10)
min_samples_list = [1, 2, 3, 4, 5]

res = {}

for min_samples in min_samples_list:
    for eps in eps_list: 
        res_df = get_dbscan_score(customer_gr, {'leaf_size':5, 'eps': eps, 'min_samples': min_samples})
        score = res_df.query('dist < @r2').shape[0]
        print(f'{eps}, {min_samples}, {score}')
        res.update({(eps, min_samples,):score})

0.001, 1, 3459
0.0016681005372000592, 1, 3473
0.0027825594022071257, 1, 3500
0.004641588833612777, 1, 3557
0.007742636826811269, 1, 3660
0.01291549665014884, 1, 3771
0.021544346900318832, 1, 3887
0.03593813663804626, 1, 3902
0.05994842503189409, 1, 3769
0.1, 1, 3468
0.001, 2, 3317
0.0016681005372000592, 2, 3338
0.0027825594022071257, 2, 3374
0.004641588833612777, 2, 3444
0.007742636826811269, 2, 3559
0.01291549665014884, 2, 3681
0.021544346900318832, 2, 3805
0.03593813663804626, 2, 3827
0.05994842503189409, 2, 3702
0.1, 2, 3407
0.001, 3, 3132
0.0016681005372000592, 3, 3152
0.0027825594022071257, 3, 3186
0.004641588833612777, 3, 3261
0.007742636826811269, 3, 3409
0.01291549665014884, 3, 3541
0.021544346900318832, 3, 3684
0.03593813663804626, 3, 3717
0.05994842503189409, 3, 3606
0.1, 3, 3325
0.001, 4, 2927
0.0016681005372000592, 4, 2950
0.0027825594022071257, 4, 3002
0.004641588833612777, 4, 3092
0.007742636826811269, 4, 3248
0.01291549665014884, 4, 3390
0.021544346900318832, 4, 3555
0.0

Наилучшие результаты получены при eps = 0.036, min_samples = 1